In [1]:
import model_lib
import numpy as np
import warnings
warnings.filterwarnings('ignore', '.*output shape of zoom.*')
import pickle
import importlib
importlib.reload(model_lib)
import os
import time
import cv2 as cv

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "2"


In [2]:
# config to train
# TODO: check Config is correct
class ProposalConfig():
    NAME = "InSegm"
    GPU_COUNT = 1
    # online training
    IMAGES_PER_GPU = 1
    STEPS_PER_EPOCH = 100
    NUM_WORKERS = 1
    PIN_MEMORY = True
    DATA_ORDER = "cw_ins"
    VALIDATION_STEPS = 20
    # including gt
    NUM_CLASSES = 81
    
    # only flips
    MEAN_PIXEL = np.array([0.485, 0.456, 0.406],dtype=np.float32).reshape(1,1,-1)
    STD_PIXEL = np.array([0.229, 0.224, 0.225],dtype=np.float32).reshape(1,1,-1)
    CLASS_NAMES = [
        'BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
        'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign',
        'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep',
        'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella',
        'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard',
        'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard',
        'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork',
        'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange',
        'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair',
        'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv',
        'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave',
        'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase',
        'scissors', 'teddy bear', 'hair drier', 'toothbrush'
    ]
    WIDTH = 224
    HEIGHT = 224
    CROP_SIZE = 224
    def __init__(self):
        self.BATCH_SIZE = self.IMAGES_PER_GPU * self.GPU_COUNT
        self.IMAGE_SHAPE = (self.WIDTH, self.HEIGHT,3)

    def display(self):
        """Display Configuration values."""
        print("\nConfigurations:")
        for a in dir(self):
            if not a.startswith("__") and not callable(getattr(self, a)):
                print("{:30} {}".format(a, getattr(self, a)))
        print("\n")

In [3]:
train_image_dir = "/media/data/nishanth/aravind/train2017/"
val_image_dir = "/media/data/nishanth/aravind/val2017/"
config = ProposalConfig()
model_dir = "./models/"
train_pickle = "/home/aravind/re/data/train_cwid.pickle"
val_pickle = "/home/aravind/re/data/val_cwid.pickle"

In [4]:
# with open(train_pickle,"rb") as train_ann:
#     train_cwid = pickle.load(train_ann)
with open(val_pickle,"rb") as val_ann:
    val_cwid = pickle.load(val_ann)

In [5]:
# train_loader = model_lib.get_loader(train_cwid,config,train_image_dir)
val_loader = model_lib.get_loader(val_cwid,config,val_image_dir)

In [ ]:
import torch
import torch.nn.functional as F
from PIL import Image
net = model_lib.SimpleHGModel()

pretrained_dict = torch.load(model_dir+"model_mask_vgg.pt")
net = model_lib.SimpleHGModel()
net_dict = net.state_dict()

pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in net_dict}
net_dict.update(pretrained_dict) 
net.load_state_dict(net_dict)

# net.vgg.load_state_dict(torch.load(model_dir+"model_vgg_class_only.pt").vgg)
# net.classifier.load_state_dict(torch.load(model_dir+"model_vgg_class_only.pt").classifier)
net = net.cuda()
with torch.no_grad():
    for i,data in enumerate(val_loader):
        batch_images,batch_impulses,batch_gt_responses,batch_class_ids = data
        print(config.CLASS_NAMES[np.argmax(batch_class_ids.numpy())])
        a,b,c= batch_images.numpy(), batch_impulses.numpy(),batch_gt_responses.numpy()
        a = np.moveaxis(a,1,-1)
        b = np.moveaxis(b,1,-1)
        c = np.moveaxis(c,1,-1)
        Image.fromarray(((a[0]*config.STD_PIXEL + config.MEAN_PIXEL)*255).astype(np.uint8),"RGB").show()
        Image.fromarray((b[0][:,:,0]*128).astype(np.uint8),"L").show()
        Image.fromarray((c[0][:,:,0]*128).astype(np.uint8),"L").show()
        batch_images,batch_impulses,batch_gt_responses,batch_class_ids = batch_images.cuda(),batch_impulses.cuda(),batch_gt_responses.cuda(),batch_class_ids.cuda()
        pred_class,pred_masks = net([batch_images,batch_impulses])
#         print(pred_class)
        pred_class = F.softmax(pred_class,dim=-1).squeeze()
        maxs, indices = torch.topk(pred_class,5,-1)
        pred_masks = (F.sigmoid(pred_masks).clamp(0.8)*255).squeeze().cpu().numpy()
        Image.fromarray(pred_masks[0].astype(np.uint8),"L").convert("RGB").show()
        Image.fromarray(pred_masks[1].astype(np.uint8),"L").convert("RGB").show()
        Image.fromarray(pred_masks[2].astype(np.uint8),"L").convert("RGB").show()
        Image.fromarray(pred_masks[3].astype(np.uint8),"L").convert("RGB").show()
#         print(maxs.shape,indices.shape)
        for i in range(5):
            print(maxs[i],indices[i],config.CLASS_NAMES[int(indices[i])])
#         print(batch_class_ids)
#         print(indices)
#         print("gt_class:",config.CLASS_NAMES[int(batch_class_ids[0])])
#         print("pred_class: ",pred_class[int(batch_class_ids[0])])
#         print(config.CLASS_NAMES[int(indices[0])])
        input()

pizza
tensor(1.00000e-02 *
       2.1192, device='cuda:0') tensor(4, device='cuda:0') motorcycle
tensor(1.00000e-02 *
       2.0328, device='cuda:0') tensor(8, device='cuda:0') truck
tensor(1.00000e-02 *
       1.9476, device='cuda:0') tensor(66, device='cuda:0') remote
tensor(1.00000e-02 *
       1.8740, device='cuda:0') tensor(31, device='cuda:0') skis
tensor(1.00000e-02 *
       1.8673, device='cuda:0') tensor(43, device='cuda:0') fork

surfboard
tensor(1.00000e-02 *
       1.9681, device='cuda:0') tensor(4, device='cuda:0') motorcycle
tensor(1.00000e-02 *
       1.9522, device='cuda:0') tensor(8, device='cuda:0') truck
tensor(1.00000e-02 *
       1.9092, device='cuda:0') tensor(66, device='cuda:0') remote
tensor(1.00000e-02 *
       1.8338, device='cuda:0') tensor(43, device='cuda:0') fork
tensor(1.00000e-02 *
       1.7896, device='cuda:0') tensor(31, device='cuda:0') skis

oven
tensor(1.00000e-02 *
       1.9943, device='cuda:0') tensor(4, device='cuda:0') motorcycle
tensor(1.000